# [Lab 11] Bokeh: quick (and partial) summary
Marco Chierici & Giuseppe Jurman

May 31, 2023

(partially abridged from the [Bokeh User Guide](https://docs.bokeh.org/en/latest/docs/user_guide.html))

**Bokeh** is a Python interactive visualization library that targets modern web browsers for presentation. Bokeh provides elegant, concise construction of novel graphics with high-performance interactivity over very large or streaming datasets in a quick and easy way.

To offer both simplicity and the powerful and flexible features needed for advanced customizations, Bokeh exposes two interface levels to users:

- a low-level bokeh.models interface that provides the most flexibility to application developers;
- an higher-level bokeh.plotting interface centered around composing visual glyphs.

To install Bokeh: `pip install bokeh` or `conda install bokeh` (`conda install -c bokeh bokeh`)


# Bokeh Glossary

**Application**: A Bokeh application is a rendered Bokeh document, running in a browser.

**BokehJS**: The JavaScript client library that actually renders the visuals and handles the UI interactions for Bokeh plots and widgets in the browser. Typically, users will not have to think about this aspect of Bokeh much (“We write the JavaScript, so you don’t have to!”) but it is good to have basic knowledge of this dichotomy. 

**Documents**: An organizing data structure for Bokeh applications. Documents contain all the Bokeh Models and data needed to render an interactive visualization or application in the browser.

**Embedding**: Various methods of including Bokeh plots and widgets into web apps and pages, or the IPython notebook.

**Glyphs**: The basic visual building blocks of Bokeh plots, e.g. lines, rectangles, squares, wedges, patches, etc. The bokeh.plotting interface provides a convenient way to create plots centered around glyphs.

**Models**: The lowest-level objects that comprise Bokeh “scenegraphs”. These live in the bokeh.models interface. Most users will not use this level of interface to assemble plots directly. However, ultimately all Bokeh plots consist of collections of models, so it is important to understand them enough to configure their attributes and properties.

**Server**: The Bokeh server is an optional component that can be used for sharing and publishing Bokeh plots and apps, for handling streaming of large data sets, or for enabling sophisticated user interactions based off of widgets and selections.

**Widgets**: User interface elements outside of a Bokeh plot such as sliders, drop down menus, buttons, etc. Events and updates from widgets can inform additional computations, or cause Bokeh plots to update. Widgets can be used in both standalone applications or with the Bokeh server.

# Getting Started

Let's begin with some examples.

Plotting data in basic Python lists as a line plot including zoom, pan, save, and other tools is simple and straightforward:

In [1]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
# prepare two lists with some data
x = [1, 2, 3, 4, 5, 6, 7]
y = [6, 7, 2, 4, 5, 10, 4]

# create a new plot with title and axis labels
p = figure(title="A simple chart", x_axis_label='x', y_axis_label='y')

# add a line renderer with color, legend, and line thickness
p.line(x, y, color="navy", legend_label="Data", line_width=2)

# show the results
show(p)

## Multiple lines

In [3]:
# prepare data
x = [1, 2, 3, 4, 5]
y1 = [6, 7, 2, 4, 5]
y2 = [2, 3, 4, 5, 6]
y3 = [4, 5, 5, 7, 2]

p = figure(title="Multiple lines", x_axis_label='x', y_axis_label='y')
p.line(x, y1, legend_label="Temp.", color="firebrick", line_width=2)
p.line(x, y2, legend_label="Rate", color="navy", line_width=2)
p.line(x, y3, legend_label="Objs.", color="darkgreen", line_width=2)
show(p)

In [4]:
p = figure(title="Multiple lines (v2)", x_axis_label='x', y_axis_label='y')

p.multi_line(xs=3*[x], ys=[y1, y2, y3],
             color=["firebrick", "navy", "darkgreen"],
             line_width=2)
show(p)

## Bokeh workflow

1. Prepare your data (e.g. plain Python lists, ...)
2. Call Bokeh's `figure()`
3. Add Bokeh renderers (e.g. `circle()`, `line()`)
4. Call Bokeh's `show()`

# Data sources

We just saw using Python lists as data sources for Bokeh.

Bokeh can also work well with NumPy arrays, Pandas series and dataframes, etc. At lower levels, these inputs are converted to a Bokeh `ColumnDataSource`. This data type is the central data source object used throughout Bokeh. Although Bokeh often creates them for us transparently, there are times when it is useful to create them explicitly.

In [5]:
from bokeh.models import ColumnDataSource

The `ColumnDataSource` is a mapping of column names (strings) to sequences of values. Here is a simple example. The mapping is provided by passing a Python `dict` with string keys and simple Python lists as values. The values could also be NumPy arrays, or Pandas sequences.

***NOTE: ALL the columns in a `ColumnDataSource` must always be the SAME length.***


## Python dictionaries

In [6]:
source = ColumnDataSource(data={
    'x' : [1, 2, 3, 4, 5],
    'y' : [3, 7, 8, 5, 1],
})

Now we pass a `ColumnDataSource` explicitly as the `source` argument to a glyph method.

Whenever we do this, if we want to map a property (like `x` or `y`) to values, we pass the ***name of that column*** in the data source:

In [7]:
p = figure(width=400, height=400)
p.circle('x', 'y', size=20, source=source)
show(p)

## Pandas DataFrames

In [8]:
from bokeh.sampledata.iris import flowers as df

source = ColumnDataSource(df)

p = figure(width=400, height=400, x_axis_label="Petal length", y_axis_label="Petal width")
p.circle('petal_length', 'petal_width', source=source)
show(p)

## Automatic Conversion

If you do not need to share data sources, it may be convenient to pass dicts, lists, or Pandas `DataFrame`s objects directly to glyph methods: in this case, a `ColumnDataSource` is created automatically.

In [9]:
from bokeh.sampledata.iris import flowers as df

p = figure(width=400, height=400, x_axis_label="Petal length", y_axis_label="Petal width")
p.circle('petal_length', 'petal_width', source=df, color='green')
show(p)

# Customizing renderers

We can use the `circle()` function instead of `line()` to render circles:

In [10]:
# create a new plot with a title and axis labels
p = figure(title="Multi-glyphs example", x_axis_label="x", y_axis_label="y")

# add multiple renderers
p.line(x, y1, legend_label="Temp.", color="firebrick", line_width=3)
p.line(x, y2, legend_label="Rate", color="navy", line_width=3)
p.circle(x, y3, legend_label="Objs.", color="darkgreen", size=16)

# show the results
show(p)

## Bar charts

Vertical and horizontal bar charts are drawn by the `vbar()` and `hbar()` renderers. Notice their syntax (with default values):

```
vbar(x, top, bottom=0, width=1)
hbar(y, right, left=0, height=1)
```

In [11]:
# create a new plot with a title and axis labels
p = figure(title="Multi-glyphs example", x_axis_label="x", y_axis_label="y")

# add multiple renderers
p.line(x, y1, legend_label="Temp.", color="firebrick", line_width=3)
p.vbar(x=x, top=y2, legend_label="Rate", color="navy", line_width=3, width=0.5)
p.circle(x, y3, legend_label="Objs.", color="darkgreen", size=16)

# show the results
show(p)

In [12]:
p = figure(title="Bar chart")

p.hbar(y=[1, 2, 3],
       right=[1.5, 2.7, 1.9],
       height=0.5)

# show the results
show(p)

Should we have categories on the y or x axis, we need to pass them as the `y_range` or `x_range` argument to `figure()`:

In [13]:
categories = ['Cat A', 'Cat B', 'Cat C']

p = figure(title="Bar chart", y_range=categories)

p.hbar(y=categories,
          right=[1.5, 2.7, 1.9],
          height=0.5)

# show the results
show(p)

## Setting and customizing glyph properties

In the previous examples, we used the `color` property to define the color of an object. `color` is an alias that automatically sets all color properties of an object to the same color, e.g. `fill_color` and `line_color`.

Other glyph properties that you can set while creating them are:

- `fill_alpha`
- `size`
- `legend_label`


In [14]:
x = [1, 2, 3, 4, 5]
y = [4, 5, 5, 7, 2]

p = figure(title="Glyphs properties")

p.circle(x, y,
         legend_label="Points",
         fill_color="orange",
         fill_alpha=0.5,
         line_color="blue",
         size=80,
        )

show(p)

It is also possible to change glyph properties after creation, provided that you first save the glyph to a variable:

In [15]:
x = [1, 2, 3, 4, 5]
y = [4, 5, 5, 7, 2]

p = figure(title="Glyphs properties")

circle = p.circle(x, y,
         legend_label="Points",
         fill_color="orange",
         fill_alpha=0.5,
         line_color="blue",
         size=80,
        )

# change properties after creation
glyph = circle.glyph
glyph.fill_color = "blue"

show(p)

# Legends and annotations

## Styling legends
Bokeh automaticall adds a legend if you use the `legend_label` argument in the renderer.

Under the hood, this adds a legend object to plot `p`, which can accessed through `p.legend` to modify its properties.

In [16]:
x = [1, 2, 3, 4, 5]
y1 = [4, 5, 5, 7, 2]
y2 = [2, 3, 4, 5, 6]

p = figure(title="Legend example")

line = p.line(x, y1, legend_label="Temp.", line_color="blue", line_width=2)
circle = p.circle(
    x,
    y2,
    legend_label="Objects",
    fill_color="red",
    fill_alpha=0.5,
    line_color="blue",
    size=80,
)

p.legend.location = "top_left" # default is "top_right"
p.legend.title = "Obervations"

# legend text
p.legend.label_text_font = "times"
p.legend.label_text_font_style = "italic"
p.legend.label_text_color = "navy"

# legend border and background
p.legend.border_line_width = 3
p.legend.border_line_color = "navy"
p.legend.border_line_alpha = 0.8
p.legend.background_fill_color = "navy"
p.legend.background_fill_alpha = 0.2

# define the click policy on the legend
p.legend.click_policy = "hide"  # "hide" or "mute"

# show the results
show(p)

## Styling titles

In [17]:
x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

p = figure(title="Title example")

p.line(x, y, legend_label="Temp.", line_width=2)

p.title_location = "left"
p.title.text = "Changing title text example"

# style the headline
p.title.text_font_size = "25px"
p.title.align = "right"
p.title.background_fill_color = "darkgrey"
p.title.text_color = "white"

# show the results
show(p)

## Adding box annotations

Box annotations can highlight certain areas of the plot.

In [18]:
from bokeh.models import BoxAnnotation

```
BoxAnnotation(top, bottom, ...)
```

If you do not pass a value for `top` or `bottom`, Bokeh automatically extends the box's dimension to the edges of the plot.

After you create box annotations, you need to add them to the existing figure with the `add_layout()` method.

In [19]:
import random

random.seed(45)

x = list(range(0, 51))
y = random.sample(range(0, 100), 51)

# new plot
p = figure(title="Box annotation")

# add line renderer
line = p.line(x, y, line_color="black", line_width=2)

# add box annotations
low_box = BoxAnnotation(top=20, fill_alpha=0.2, fill_color="yellow")
mid_box = BoxAnnotation(bottom=20, top=80, fill_alpha=0.2, fill_color="lime")
high_box = BoxAnnotation(bottom=80, fill_alpha=0.2, fill_color="yellow")

# # add boxes to existing figure
p.add_layout(low_box)
p.add_layout(mid_box)
p.add_layout(high_box)

# show the results
show(p)

# Customizing plots

## Themes

Bokeh has some built-in themes: `caliber`, `dark_minimal`, `light_minimal`, `night_sky`, and `contrast`.

The themes are applied to the current document by importing and using Bokeh's `curdoc()`.

In [20]:
from bokeh.io import curdoc

x = [1, 2, 3, 4, 5]
y = [4, 5, 5, 7, 2]

# set theme for current document
curdoc().theme = "dark_minimal"

# create a plot
p = figure(title="Using themes")

# add a renderer
p.line(x, y)

# show the results
show(p)

## Plot size

In [21]:
curdoc().theme = 'caliber' # reset to default theme

p = figure(title="Plot size",
           x_axis_label='x',
           y_axis_label='y',
           width=350,
           height=250,
          )

p.circle(x, y, color="red", size=15)

show(p)

As we did before with glyph properties, we can change plot attributes, including size, after creation:

In [22]:
curdoc().theme = 'caliber' # reset to default theme

p = figure(title="Plot size",
           x_axis_label='x',
           y_axis_label='y',
           width=350,
           height=250,
          )

p.width = 450
p.height = 150

p.circle(x, y, color="red", size=15)

show(p)

Plots can also be made responsive to automatically adjust to the browser or screen size, with the attribute `sizing_mode`:

In [23]:
p = figure(title="Responsive plot",
           x_axis_label='x',
           y_axis_label='y',           
           sizing_mode='stretch_width',
           height=250,
           max_width=700,
          )

p.circle(x, y, color='red', size=15)
show(p)

## Custom axes

### Appearance

How-to: set different properties of the `xaxis()` and `yaxis()` methods of the Plot object.

In [24]:
x = [1, 2, 3, 4, 5]
y = [4, 5, 5, 7, 2]

p = figure(
    title="Customized axes",
    width=450,
    height=350,
)

p.circle(x, y, size=10)

# change some things about the x-axis
p.xaxis.axis_label = "Temp"
p.xaxis.axis_line_width = 3
p.xaxis.axis_line_color = "red"

# change some things about the y-axis
p.yaxis.axis_label = "Pressure"
p.yaxis.major_label_text_color = "orange"
p.yaxis.major_label_orientation = "vertical"

# change things on all axes
p.axis.minor_tick_in = -3
p.axis.minor_tick_out = 6

# show the results
show(p)

### Ranges

How-to: use the `y_range` or `x_range` properties of the Plot object when you call `figure()`.

In [25]:
p = figure(
    title="Customized ranges",
    y_range=(0, 25),
    width=450,
    height=350,
)

p.circle(x, y, size=10)
show(p)

### Axis tick labels

You can format these labels with a `TickFormatter` object, i.e., a `NumeralTickFormatter`:

In [26]:
from bokeh.models import NumeralTickFormatter

p = figure(title='Tick formatter',
           width=450,
           height=350,
          )

p.circle(x, y, size=8)
p.line(x, y, color="blue")

p.yaxis[0].formatter = NumeralTickFormatter(format="$0.00")

show(p)

Another useful formatter is `PrintfTickFormatter`, which allows you to specify a custom format string.

Bokeh's default formatters are `CategoricalTickFormatter`, `DatetimeTickFormatter`, `LogTickFormatter`, and `BasicTickFormatter`, but they don't provide many customizations.

In [27]:
from bokeh.models import PrintfTickFormatter

p = figure(title='Tick formatter',
           width=450,
           height=350,
          )

p.circle(x, y, size=8)
p.line(x, y, color="blue")

p.yaxis[0].formatter = NumeralTickFormatter(format="$0.00")
p.xaxis[0].formatter = PrintfTickFormatter(format="%.1f")

show(p)

### Logscale

How-to: use `y_axis_type="log"` or `x_axis_type="log"` to switch to logscale.

In [28]:
x = [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
y0 = [i**2 for i in x]
y1 = [10**i for i in x]
y2 = [10**(i**2) for i in x]

# create a new plot with a logarithmic axis type
p = figure(
    title="Logarithmic axis example",
    y_axis_type="log",
    y_range=[0.001, 1e11],
    x_axis_label="sections",
    y_axis_label="particles",
)

# add some renderers
p.line(x, x, legend_label="y=x")
p.circle(x, x, legend_label="y=x", fill_color="white", size=8)
p.line(x, y0, legend_label="y=x^2", line_width=3)
p.line(x, y1, legend_label="y=10^x", line_color="red")
p.circle(x, y1, legend_label="y=10^x", fill_color="red", line_color="red", size=6)
p.line(x, y2, legend_label="y=10^x^2", line_color="orange", line_dash="4 4")

p.legend.location = "top_left"
show(p)

### Datetime

How-to: set the `x_axis_type` or `y_axis_type` to `"datetime"`

In [29]:
from bokeh.models import DatetimeTickFormatter, NumeralTickFormatter
from datetime import datetime, timedelta

# generate list of dates (today's date in subsequent weeks)
dates = [(datetime.now() + timedelta(day * 7)) for day in range(0, 26)]
# generate 25 random data points
y = random.sample(range(0, 100), 26)

p = figure(title="datetime axis example",
           x_axis_type="datetime",
           height=250)

# add renderers
p.circle(dates, y, size=8)
p.line(dates, y, color="navy", line_width=1)

# format axes ticks
p.yaxis[0].formatter = NumeralTickFormatter(format="$0.00")
p.xaxis[0].formatter = DatetimeTickFormatter(months="%b %Y")

# show the results
show(p)

## Grid

How-to: set different properties of the `xgrid()`, `ygrid()`, and `grid()` methods of the Plot object.

In [30]:
x = [1, 2, 3, 4, 5]
y = [4, 5, 5, 7, 2]

# create a plot
p = figure(title="Custom grid lines", height=250)

# add a renderer
p.line(x, y, line_color="darkgreen", line_width=2)

# change only the x-grid
p.xgrid.grid_line_color = "red"

# change only the y-grid
p.ygrid.grid_line_alpha = 0.8
p.ygrid.grid_line_dash = [6, 4] # length, spacing

# show the results
show(p)

## Customizing plot fill color

In [31]:
x = [1, 2, 3, 4, 5]
y = [4, 5, 5, 7, 2]

p = figure(title="Custom plot colors", height=250)
p.line(x, y, line_color="darkgreen", line_width=2)

# change fill colors
p.background_fill_color = "#f7f7f7"
p.border_fill_color = "#cccccc"
p.outline_line_color = "#ff0000"

# show the results
show(p)

## Customizing the toolbar

Not only can you change the toolbar position, but also customize the available tools. Let's deal with the toolbar position first:

In [32]:
p = figure(title="Toolbar position", height=250,
           toolbar_location="below")

p.line(x, y, line_color="darkgreen", line_width=2)

show(p)

You can also change the toolbar location after you create the plot:

In [33]:
p = figure(title="Toolbar position", height=250)
p.toolbar_location = "below"

p.line(x, y, line_color="darkgreen", line_width=2)

show(p)

You can deactivate the toolbar, or set it to hide automatically (i.e. it is shown only if the mouse is inside the plot area).

In [34]:
p = figure(title="Toolbar position", height=250)
# p.toolbar_location = None
# p.toolbar.autohide = True

p.line(x, y, line_color="darkgreen", line_width=2)

show(p)

You can customize the tools shown in the toolbar using the `tools` argument within `figure()`.

In [35]:
p = figure(title="Toolbar tools", height=250,
           tools="pan,box_zoom,reset,save")

p.line(x, y, line_color="darkgreen", line_width=2)

show(p)

For more customization, instead of a comma-separated string containing tool shortcut names you can pass a list of tool objects (but you need to import these first):

In [36]:
from bokeh.models import BoxZoomTool, PanTool, ResetTool

p = figure(title="Toolbar tools v2", height=250,
           tools=[BoxZoomTool(), ResetTool()])

# add & customize a tool afterwards
p.add_tools(PanTool(dimensions='width'))

p.circle(x, y, size=15)

show(p)

## Tooltips

Tooltips are handled by `HoverTool`: you can include it among the available tools of the `figure()` call, together with the `tooltips` argument. This argument has its own syntax: use the "@" symbol to include the name of the data source that you want Bokeh to display in the tooltip.

In [37]:
from bokeh.models import HoverTool

p = figure(
    y_range=(0, 10),
    tools=[BoxZoomTool(), HoverTool(), ResetTool()],
    tooltips="Data point @x has the value @y",
    height=250,
)

# add renderers
p.circle(x, y, size=10)
p.line(x, y, line_width=2)

# show the results
show(p)

# Combining plots

## Grid layouts

In Bokeh it is pretty easy to combine individual plots into rows, columns, and grids.

First you create the plots as distinct `figure`s, then you arrange them with `row()` or `column()` before calling `show()`.

In [38]:
from bokeh.layouts import row, column
import numpy as np

X = np.linspace(-5, 5, 100)
y0 = X
y1 = X**2
y2 = X**3

s1 = figure(width=250, height=250, toolbar_location=None)
s1.line(X, y0)

s2 = figure(width=250, height=250, toolbar_location=None)
s2.line(X, y1)

s3 = figure(width=250, height=250, toolbar_location=None)
s3.line(X, y2)

show(row(s1, s2, s3))
# show(column(s1, s2, s3))

In [39]:
from bokeh.layouts import gridplot
grid = gridplot([[s1, s2], [None, s3]], width=400, height=400)
# or pass a plain list specifying ncols or nrows:
# grid = gridplot([s1, s2, s3], ncols=2, width=400, height=400, toolbar_location='right')
show(grid)

# Filtered views

You can plot a filtered subset of your data using a "view" through the `CDSView` class, in which you set a `filter` property. The simplest filters are the `IndexFilter` and the `GroupFilter` (for categorical data).

In [40]:
from bokeh.models import CDSView, ColumnDataSource, IndexFilter, GroupFilter

source = ColumnDataSource(data=dict(x=[1, 2, 3, 4, 5], y=[1, 2, 3, 4, 5]))
view = CDSView(filter=IndexFilter([0, 2, 4]))

fig_opts = dict(width=300, height=300)
p1 = figure(title="Full data", **fig_opts)
p1.circle(x='x', y='y', size=10, source=source)

p2 = figure(title="Filtered view", **fig_opts)
p2.circle(x='x', y='y', size=10, source=source, view=view)

show(gridplot([[p1, p2]]))

In [41]:
from bokeh.sampledata.iris import flowers as df

source = ColumnDataSource(df)
view = CDSView(filter=GroupFilter(column_name='species', group='versicolor'))

fig_opts = dict(width=300, height=300, tools="box_select,reset,help")

p1 = figure(title="Full data", **fig_opts)
p1.circle(x='petal_length', y='petal_width', source=source)

p2 = figure(title="Filtered (versicolor)", **fig_opts)
p2.circle(x='petal_length', y='petal_width', source=source, view=view, color='firebrick')

show(row(p1, p2))

# Linked plots

In Bokeh, plot linking is typically accomplished by sharing some plot component between plots. Below is an example that demonstrates **linked panning** (where changing the range of one plot causes others to update) by sharing range objects between the plots. 

In this example, also notice the new glyph methods `Figure.triangle` and `Figure.square`.

In [42]:
# prepare some data
N = 100
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)
y2 = np.sin(x) + np.cos(x)

# create a new plot
s1 = figure()
s1.circle(x, y0, size=3, color="navy", alpha=0.5)

# create a new plot and share both ranges
s2 = figure(x_range=s1.x_range, y_range=s1.y_range)
s2.triangle(x, y1, size=3, color="firebrick", alpha=0.5)

# create a new plot and share only one range
s3 = figure(x_range=s1.x_range)
s3.square(x, y2, size=3, color="olive", alpha=0.5)

# put the subplots in a gridplot
p = gridplot([[s1, s2, s3]], toolbar_location="below", width=250, height=250)

# show the results
show(p)

Another linkage that is often useful is **linked brushing**, where a selection on one glyph causes a selection to update on all other glyphs that share the same source. The Iris plot we saw earlier already demonstrates linked brushing: here it is again, this time with a `lasso_select` tool that better highlights this kind of linkage.

In [43]:
from bokeh.sampledata.iris import flowers as df

source = ColumnDataSource(df)
view = CDSView(filter=GroupFilter(column_name='species', group='versicolor'))

fig_opts = dict(width=300, height=300, tools="lasso_select,box_select,reset,help")

p1 = figure(title="Full data", **fig_opts)
p1.circle(x='petal_length', y='petal_width', source=source)

p2 = figure(title="Filtered (versicolor)", **fig_opts)
p2.circle(x='petal_length', y='petal_width', source=source, view=view, color='firebrick')


show(row(p1, p2))

# Widgets

Widgets are interactive controls that can be added to Bokeh applications to provide a front end user interface to a visualization. They can drive new computations, update plots, and connect to other programmatic functionality.

Bokeh offers a variety of widgets, such as buttons, sliders, checkboxes, etc.

Let's create an interactive scatterplot that displays a message through a `Div` widget and allows the user to:

- change the size of the points through a `Spinner` widget;
- adjust the x-axis range through a `RangeSlider` widget.

The values generated by the Spinner and RangeSlider widgets need to be linked to the existing plot: to this aim, we use the `js_link()` function, which in turn relies on JavaScript to interactively link two Bokeh models.

In [44]:
from bokeh.layouts import layout
from bokeh.models import Div, RangeSlider, Spinner
from bokeh.plotting import figure, show

# prepare some data
x = list(range(10))
y = [4, 5, 5, 7, 2, 6, 4, 9, 1, 3]

# create plot with circle glyphs
p = figure(x_range=(1, 9), width=500, height=250)
points = p.circle(x=x, y=y, size=30, fill_color="#21a7df")

# set up textarea (div)
div = Div(
    text="""
          <p>Select the circle's size using this control element:</p>
          """,
    width=200,
    height=30,
)

# set up spinner
spinner = Spinner(
    title="Circle size",
    low=0,
    high=60,
    step=5,
    value=points.glyph.size,
    width=200,
)
spinner.js_link("value", points.glyph, "size")

# set up RangeSlider
range_slider = RangeSlider(
    title="Adjust x-axis range",
    start=0,
    end=10,
    step=1,
    value=(p.x_range.start, p.x_range.end),
)
range_slider.js_link("value", p.x_range, "start", attr_selector=0)
range_slider.js_link("value", p.x_range, "end", attr_selector=1)

# create layout
layout = layout(
    [
        [div, spinner],
        [range_slider],
        [p],
    ]
)

# show result
show(layout)